In [12]:
import do_mpc
from casadi import *

# Dynamic Model

This function defines the mathematical model for the toggle switch in the form of differential equations (ODE).

Defining $mRNA\_LacI$ the concentration of the mRNA associated with the _lacI gene_, $mRNA\_TetR$ the concentration of the mRNA associated with the _TetR gene_, $LacI$ the concentration of _LacI_ and $TetR$ the concentration of _TetR_, the system can be described via the following set of ODEs:

\begin{equation*}
    \begin{split}
        &\dot{mRNA_LacI} = k^{m_0}_L + k^m_L \frac{1}{1 + \left(\frac{TetR}{\theta_{TetR}} \frac{1}{1 + \left(\frac{v_1}{\theta_{aTc}}\right)^{\eta_{aTc}}}\right)^{\eta_{TetR}}} - g^m_L mRNA\_LacI\\
        &\dot{mRNA_TetR} = k^{m_0}_T + k^m_T \frac{1}{1 + \left(\frac{LacI}{\theta_{LacI}} \frac{1}{1 + \left(\frac{v_2}{\theta_{IPTG}}\right)^{\eta_{IPTG}}}\right)^{\eta_{LacI}}} - g^m_T mRNA\_TetR\\
        &\dot{LacI} = k^p_L mRNA\_LacI - g^p_L LacI\\
        &\dot{TetR} = k^p_T mRNA\_TetR - g^p_T TetR\\ \\
        &\dot{v_1} = \begin{cases} k^{in}_{aTc}(aTc - v_1) \quad if \quad  aTc > v_1\\ k^{out}_{aTc}(aTc - v_1) \quad if aTc \leq v_1 \end{cases}\\
        &\dot{v_2} = \begin{cases} k^{in}_{IPTG}(IPTG - v_2) \quad if \quad IPTG > v_2\\ k^{out}_{IPTG}(IPTG - v_2) \quad if IPTG \leq v_2 \end{cases}
    \end{split}
\end{equation*}

whre $aTc$ and $IPTG$ are extra-cellular concentrations of aTc and IPTg, respectively, and $v_1$ and $v_2$ are intra-cellular concentrations of aTc and IPTg, respectively.

The model parameters are:
<center>

| Parameter  | Value| Meaning|
|------------  |---------------|-----------------------------------------  |
|$k\_m0\_L    $| $3.20e^{-2}$  | Basal expression level of the LacI      |
|$k\_m0\_T    $| $1.19e^{-1}$  | Basal expression level of the TetR      |
|$k\_m\_L     $| $8.30$        | Maximal transcription rate of LacI mRNA |
|$k\_m\_T     $| $2.06$        | Maximal transcription rate TetR mRNA    |
|$k\_p\_L     $| $9.726e^{-1}$ | Rate of production of LacI              |
|$k\_p\_T     $| $9.726e^{-1}$ | Rate of production of TetR              |
|$g\_m\_L     $| $1.386e^{-1}$ | Degradation rate of LacI mRNA           |
|$g\_m\_T     $| $1.386e^{-1}$ | Degradation rate of TetR mRNA           |
|$g\_p\_L     $| $1.65e^{-2}$  | Degradation rate of LacI                |
|$g\_p\_T     $| $1.65e^{-2}$  | Degradation rate of TetR                |
|$theta\_LacI $| $31.94$       | Dissociation constants of LacI          |
|$theta\_TetR $| $30.00$       | Dissociation constants of TetR          |
|$theta\_IPTG $| $9.06e^{-2}$  | Dissociation constants of IPTG          |
|$theta\_aTc  $| $11.65$       | Dissociation constants of aTC           |
|$eta\_LacI   $| $2.00$        | Hill coefficient of LacI                |
|$eta\_TetR   $| $2.00$        | Hill coefficient of TetR                |
|$eta\_IPTG   $| $2.00$        | Hill coefficient of IPTG                |
|$eta\_aTc    $| $2.00$        | Hill coefficient of aTc                 |
|$k\_in\_aTc  $| $2.75e^{-2}$  |                                         |
|$k\_out\_aTc $| $2.00e^{-2}$  |                                         |
|$k\_in\_IPTG $| $1.62e^{-1}$  |                                         |
|$k\_out\_IPTG$| $1.11e^{-1}$  |                                         |
    



In [13]:
def template_model(stochasticity=False):

    model = do_mpc.model.Model(model_type='continuous')

    # Model states
    mRNA_LacI = model.set_variable(var_type='states', var_name='mRNA_LacI')
    mRNA_TetR = model.set_variable(var_type='states', var_name='mRNA_TetR')
    lacI = model.set_variable(var_type='states', var_name='LacI')
    tetR = model.set_variable(var_type='states', var_name='TetR')
    v1 = model.set_variable(var_type='states', var_name='v1')
    v2 = model.set_variable(var_type='states', var_name='v2')

    # Model inputs
    aTc = model.set_variable(var_type='inputs', var_name='aTc')
    iptg = model.set_variable(var_type='inputs', var_name='IPTG')

    # Model parameters
    k_m0_L = 3.20e-2
    k_m0_T = 1.19e-1
    k_m_L = 8.30
    k_m_T = 2.06
    k_p_L = 9.726e-1
    k_p_T = 9.726e-1
    g_m_L = 1.386e-1
    g_m_T = 1.386e-1
    g_p_L = 1.65e-2
    g_p_T = 1.65e-2
    theta_LacI = 31.94
    theta_TetR = 30.00
    theta_IPTG = 9.06e-2
    theta_aTc = 11.65
    eta_LacI = 2.00
    eta_TetR = 2.00
    eta_IPTG = 2.00
    eta_aTc = 2.00
    k_in_aTc = 2.75e-2
    k_out_aTc = 2.00e-2
    k_in_IPTG = 1.62e-1
    k_out_IPTG = 1.11e-1

    # Model references
    LacI_ref = 750
    TetR_ref = 300

    # Defining model's equations
    model.set_rhs('mRNA_LacI', k_m0_L + k_m_L*(1 / (1 + ((tetR/theta_TetR) * (1 /
                  (1 + (v1/theta_aTc)**eta_aTc)))**eta_TetR)) - g_m_L * mRNA_LacI, process_noise=stochasticity)
    model.set_rhs('mRNA_TetR', k_m0_T + k_m_T*(1 / (1 + ((lacI/theta_LacI) * (1 /
                  (1 + (v2/theta_IPTG)**eta_IPTG)))**eta_LacI)) - g_m_T * mRNA_TetR, process_noise=stochasticity)
    model.set_rhs('LacI', k_p_L * mRNA_LacI - g_p_L *
                  lacI, process_noise=stochasticity)
    model.set_rhs('TetR', k_p_T * mRNA_TetR - g_p_T *
                  tetR, process_noise=stochasticity)
    model.set_rhs('v1', (k_in_aTc * (aTc - v1)) * (aTc > v1) +
                  (k_out_aTc * (aTc - v1)) * (aTc <= v1), process_noise=stochasticity)
    model.set_rhs('v2', (k_in_IPTG * (iptg - v2)) * (iptg > v2) +
                  (k_out_IPTG * (iptg - v2)) * (iptg <= v2), process_noise=stochasticity)

    # The process noise w is used to simulate a disturbed system in the Simulator

    # Measurement noise
    if stochasticity:
        model.n_v = np.random.randn(6, 1)

    # Cost function
    model.set_expression(expr_name='cost', expr=(
        lacI - LacI_ref)**2 + (tetR - TetR_ref)**2)    

    model.setup()

    return model


# MPC Controller

This function configure and setup the MPC controller, given the model previously defined.

Going into detail, it defines the cost function and the constraints on state variables and inputs.

In [14]:
def template_mpc(model, setup_mpc):

    mpc = do_mpc.controller.MPC(model)

    mpc.set_param(**setup_mpc)

    # Cost function
    mterm = model.aux['cost']
    lterm = model.aux['cost']

    mpc.set_objective(mterm=mterm, lterm=lterm)
    mpc.set_rterm(aTc=1, IPTG=1)

    # Constraints
    mpc.bounds['lower', '_x', 'mRNA_LacI'] = 3.20e-2
    mpc.bounds['lower', '_x', 'mRNA_TetR'] = 1.19e-1

    mpc.bounds['lower', '_x', 'LacI'] = 0
    mpc.bounds['lower', '_x', 'TetR'] = 0

    mpc.bounds['lower', '_x', 'v1'] = 0
    mpc.bounds['lower', '_x', 'v2'] = 0

    mpc.bounds['lower', '_u', 'aTc'] = 0
    mpc.bounds['upper', '_u', 'aTc'] = 35

    mpc.bounds['lower', '_u', 'IPTG'] = 0
    mpc.bounds['upper', '_u', 'IPTG'] = 0.35

    mpc.setup()

    return mpc


# Simulator

This function configure and setup the simulator, necessary for a first testing phase of the developed control approach.

In [15]:
def template_simulator(model, t_step):

    simulator = do_mpc.simulator.Simulator(model)
    simulator.set_param(t_step=t_step)

    simulator.setup()

    return simulator

# Simulation parameters

| Parameter  | Value| Meaning|
|-------------|------|--------|
|$total\_time$|86400 s (24 h)| Total simulation time|
|$t\_step$|60s (1m)|Sample time|
|$steps$|1440|Number of iterations of the control loop|
|$avg\_period$|14400 s (240m)|Period for calculating the average trajectory of the outputs|


In [22]:
total_time = 86400  # Total simulation time 24h (86400s)
t_step = 60  # Read states every 1m (60s)
steps = int(total_time/t_step)  # Number of control step total_time/t_step 1440
avg_period = 14400  # Get average trajectory every 240 m (14400s)

# Initial conditions

In [23]:
x_0 = np.array([30, 50, 400, 125, 0, 0])